In [16]:
import torch
from torch.nn import Linear, Sequential

# from torch.distributions import Categorical
from tensordict import TensorDict
from tensordict.nn import TensorDictModule, ProbabilisticTensorDictSequential
from tensordict.nn.distributions import NormalParamExtractor
from torchrl.modules import (
    ProbabilisticActor,
    ValueOperator,
    TanhNormal,
    ActorValueOperator,
)
from torchrl.objectives import PPOLoss, ValueEstimators
from torchrl.envs import TransformedEnv, DoubleToFloat
from torchrl.envs.libs.gym import GymEnv
from torchrl.collectors import SyncDataCollector

env = TransformedEnv(
    GymEnv("InvertedDoublePendulum-v4"), DoubleToFloat(in_keys=["observation"])
)

common = TensorDictModule(Linear(11, 10), in_keys=["observation"], out_keys=["hidden"])
actor_head = ProbabilisticActor(
    module=TensorDictModule(
        Sequential(
            Linear(10, 2),
            NormalParamExtractor(),
        ),
        in_keys=["hidden"],
        out_keys=["loc", "scale"],
    ),
    in_keys=["loc", "scale"],
    distribution_class=TanhNormal,
)
value_head = ValueOperator(Linear(10, 1), in_keys=["hidden"], out_keys=["state_value"])
model = ActorValueOperator(common, actor_head, value_head)
collector = SyncDataCollector(
    env,
    model.get_policy_operator(),
    frames_per_batch=1000,
    total_frames=10000,
    split_trajs=False,
)

loss_module = PPOLoss(model.get_policy_operator(), model.get_value_operator())
loss_module.make_value_estimator(ValueEstimators.GAE, gamma=0.99, lmbda=0.95)
gae = loss_module.value_estimator

tensordict_data = next(iter(collector))

with torch.no_grad():
    gae(
        tensordict_data,
        params=loss_module.critic_params,
        target_params=loss_module.target_critic_params,
    )

/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_space` for environment variables or `env.get_wrapper_attr('reward_space')` that will search the reminding wrappers.
  logger.warn(


TensorDict(
    fields={
        observation: Tensor(shape=torch.Size([1000, 11]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([1000]),
    device=cpu,
    is_shared=False)
TensorDict(
    fields={
        observation: Tensor(shape=torch.Size([1000, 11]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([1000]),
    device=cpu,
    is_shared=False)
